In [2]:
import base64
import datetime
import json
import os
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

We found student's assessment data for the state of california.  The data contains summary columns and detail columns.
It contains state summary data, County summary data and District summary data as well as detailed per school.
Since our analysis will be done at the District level, we created a script to read the assessment data and extract only the district summary data.

Which meant removing state and county data as well as school detailed data.
The file has 3 main columns that function as index.
County Code   Distric Code  School Code
00            00000         0000000       - This is the State Data
01 - 58       00000         0000000       - This is the County Summary Data
01-58         Various       0000000       - This is the District data which we are interested in
01-58         Various       Various       - This is the data per school

In [ ]:
file1 = open('commondata/sb_ca2017_all_csv_v2.txt', 'r') 
Lines = file1.readlines()
file2 = open('cleaneddata.txt', 'w') 
for x in range(0,3264975):
    #pdb.set_trace()
    #pdb.set_trace()
    if "County Code" in Lines[x].split(",")[0]:
        print(Lines[x])
        file2.writelines(Lines[x]) 
    elif '"00"' == Lines[x].split(",")[0]:
        continue
    elif '"00000"' == Lines[x].split(",")[1]:
        #then skip
        continue
    elif '"0000000"' != Lines[x].split(",")[2]:
        continue
        #then skip
        #file2.writelines(Lines[x])
        #print(Lines[x])
    #print(Lines[x])
    file2.writelines(Lines[x]) 

Afterwards we created a data frame with the new cleaned data.
The data we have for assessment has information on the Grades and also has a subgroup code which has ethnic and other groups information which we will need to separate.

In [3]:

# Read the data file and take a look at the data
df = pd.read_csv('cleaneddata.txt', sep=",")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,4,5,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [204]:
df.head(100)

,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0
7,1,10017,1,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0
8,1,10017,1,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0
9,1,10017,1,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0


In [5]:
#We will drop the first row since it is duplicating the column names
df.drop([0], axis=0, inplace=True)

In [6]:
#We will drop the filler column, this column has no information, it is only blank spaces
df.drop(['Filler'], axis=1 , inplace = True)

In [7]:
#since we have district summary data, the school code of 0000000 is not useful, and we will remove that column
df.drop(['School Code'], axis=1, inplace = True)

In [9]:
df.columns

Index(['County Code', 'District Code', 'Test Year', 'Subgroup ID', 'Test Type',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores', 'Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'],
      dtype='object')

In [10]:
df.drop(['Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'], axis=1, inplace=True)

In [11]:
df.columns

Index(['County Code', 'District Code', 'Test Year', 'Subgroup ID', 'Test Type',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores'],
      dtype='object')

In [12]:
df.drop(['Test Year','Test Type','Total Tested At Entity Level','Total Tested with Scores'], axis=1, inplace=True)

In [13]:
#then we will reset the index and drop index column
df=df.reset_index()
df.head()

,index,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,01,10017,1,6,2,*,*,*,*,*,*,*,*,*
1,2,01,10017,1,6,1,*,*,*,*,*,*,*,*,*
2,3,01,10017,1,7,2,*,*,*,*,*,*,*,*,*
3,4,01,10017,1,7,1,*,*,*,*,*,*,*,*,*
4,5,01,10017,1,8,1,*,*,*,*,*,*,*,*,*


In [14]:
df.drop(['index'], axis = 1, inplace=True)
df.head()

,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,01,10017,1,6,2,*,*,*,*,*,*,*,*,*
1,01,10017,1,6,1,*,*,*,*,*,*,*,*,*
2,01,10017,1,7,2,*,*,*,*,*,*,*,*,*
3,01,10017,1,7,1,*,*,*,*,*,*,*,*,*
4,01,10017,1,8,1,*,*,*,*,*,*,*,*,*


In [174]:
df=df.apply(pd.to_numeric, errors='coerce')

In [16]:
#when we run the df Grade unique code we saw that there were grades in text and others in numbers, so we replaced the values to be all numbers

#df['Grade'].replace(to_replace ="6", value =6, inplace=True) 
#df['Grade'].replace(to_replace ="7", value =7, inplace=True)
#df['Grade'].replace(to_replace ="8", value =8, inplace=True)
#df['Grade'].replace(to_replace ="11", value =11, inplace=True)
#df['Grade'].replace(to_replace ="4", value =4, inplace=True)
#df['Grade'].replace(to_replace ="5", value =5, inplace=True)
#df['Grade'].replace(to_replace ="3", value =3, inplace=True)
#df['Grade'].replace(to_replace ="13", value =13, inplace=True)



In [175]:
df['Grade'].unique()

array([ 6,  7,  8, 11, 13,  4,  5,  3])

In [123]:
df.astype({'Grade': 'int64'}).dtypes

County Code                          object
District Code                        object
Subgroup ID                          object
Grade                                 int64
Test Id                              object
CAASPP Reported Enrollment           object
Students Tested                      object
Mean Scale Score                     object
Percentage Standard Exceeded         object
Percentage Standard Met              object
Percentage Standard Met and Above    object
Percentage Standard Nearly Met       object
Percentage Standard Not Met          object
Students with Scores                 object
dtype: object

In [176]:
df['Grade'].dtype

dtype('int64')

In [125]:
df.astype({'Subgroup ID': 'int64'}).dtypes

County Code                          object
District Code                        object
Subgroup ID                           int64
Grade                                 int64
Test Id                              object
CAASPP Reported Enrollment           object
Students Tested                      object
Mean Scale Score                     object
Percentage Standard Exceeded         object
Percentage Standard Met              object
Percentage Standard Met and Above    object
Percentage Standard Nearly Met       object
Percentage Standard Not Met          object
Students with Scores                 object
dtype: object

In [177]:
df['Subgroup ID'].dtype

dtype('int64')

In [178]:
df['Subgroup ID'].unique()

array([  1,   3,   4,   6,   7,   8,  31,  74,  76,  77,  78,  79,  80,
        90,  91,  92,  93,  94,  99, 111, 121, 128, 142, 144, 160, 170,
       180, 200, 202, 204, 205, 206, 207, 220, 222, 223, 224, 225, 226,
       227, 120, 203,  75, 201, 221, 190,  28])

In [23]:
#Subgroup ID 1 is All students. We will correct the subgroup_id to show only one type of value, since we see text and numeric
#df['Subgroup ID'].replace(to_replace =1 , value ='1', inplace=True) #will be making it text since it is a category.

In [179]:
df['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [129]:
df['County Code'].replace(to_replace ="01", value =1, inplace=True)
df['County Code'].replace(to_replace = "02", value=2, inplace=True)
df['County Code'].replace(to_replace ="03", value =3, inplace=True)
df['County Code'].replace(to_replace ="04", value =4, inplace=True) 

In [205]:
df.head(29)

,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0
7,1,10017,1,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0
8,1,10017,1,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0
9,1,10017,1,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0


In [25]:
df_subgroups= pd.read_csv('Subgroups.txt', sep=',')
df_subgroups = df_subgroups.set_index('id')

In [26]:
df_subgroups

,subgroup_id,subgroup_definition,subgroup_type
id,,,
1,1,"""All Students""","""All Students"""
3,3,"""Males""","""Gender"""
4,4,"""Females""","""Gender"""
6,6,"""Fluent-English Proficient and English Only""","""English-Language Fluency"""
7,7,"""Initially-Fluent English Proficient (IFEP)""","""English-Language Fluency"""
8,8,"""Reclassified-Fluent English Proficient (RFEP)""","""English-Language Fluency"""
28,28,"""Education""","""Migrant"""
31,31,"""EconomicallMigranty Disadvantaged""","""Economic Status"""
74,74,"""Black or African American""","""Ethnicity"""


In [181]:
df['Students Tested'].unique()

array([   nan,    44.,    38., ...,  3854.,  4131.,  4366.])

In [182]:
df.groupby(df['County Code']).size()

County Code
1     12541
2       290
3      1040
4      6578
5      2130
6      2144
7     10292
8      1082
9      6636
10    15555
11     3704
12    10790
13     6753
14     2580
15    21299
16     6308
17     3544
18     3466
19    50062
20     4736
21     7504
22      666
23     5451
24     9634
25     1504
26     1120
27    10696
28     2582
29     3960
30    16763
31     8127
32      814
33    16310
34     8754
35     3408
36    20085
37    22316
38     1208
39     9331
40     5836
41    11972
42     8946
43    17001
44     5076
45     9813
46      420
47     7219
48     4510
49    16979
50    13464
51     5116
52     5428
53     2470
54    18186
55     4098
56    10503
57     3590
58     2600
dtype: int64

In [184]:
df_math_assessment = df[df['Test Id']==2]
df_litheracy_assessment = df[df['Test Id']==1]

In [185]:
df_math_assessment['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [186]:
df_litheracy_assessment['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [189]:
df_all = df[df['Subgroup ID']==1]


In [160]:
df_tests= pd.read_csv('Tests.txt', sep=',')
df_tests = df_tests.set_index('ID')

In [161]:
df_tests.astype({'Test ID Num': 'object'}).dtypes

Test ID Num    object
Test Name      object
dtype: object

In [190]:
df['Test Id'].unique()

array([2, 1])

In [35]:
#df['Test Id'].replace(to_replace =1 , value ='1', inplace=True)
#df['Test Id'].replace(to_replace =2 , value ='2', inplace=True)

In [191]:
df['Test Id'].unique()

array([2, 1])

In [192]:
df['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [193]:
df_math=df_math_assessment.reset_index()

In [195]:
df_math=df_math[df_math['Subgroup ID']==1]


In [196]:
df_math.head()

,index,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,0,1,10017,1,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,10017,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,1,10017,1,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,1,10017,1,11,2,151.0,44.0,2412.8,0.0,0.0,0.0,4.55,95.45,44.0
4,8,1,10017,1,13,2,177.0,51.0,NaN,0.0,0.0,0.0,3.92,96.08,51.0


In [197]:
df_math.drop(['index','Subgroup ID','Test Id'], axis=1, inplace=True)

In [198]:
df_math['Grade'].unique()

array([ 6,  7,  8, 11, 13,  4,  5,  3])

In [199]:
df_math.columns

Index(['County Code', 'District Code', 'Grade', 'CAASPP Reported Enrollment',
       'Students Tested', 'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores'],
      dtype='object')

In [202]:
#df_math['ns_exceeded']=df_math['Percentage Standard Exceeded']*df_math['Students Tested']
df_math['ns_met']=df_math['Percentage Standard Met']*df_math['Students Tested']
df_math['ns_metaabove']=df_math['Percentage Standard Met and Above']*df_math['Students Tested']
df_math['ns_nmet']=df_math['Percentage Standard Nearly Met']*df_math['Students Tested']
df_math['ns_notmet']=df_math['Percentage Standard Not Met']*df_math['Students Tested']

In [203]:
df_math.columns

Index(['County Code', 'District Code', 'Grade', 'CAASPP Reported Enrollment',
       'Students Tested', 'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores', 'ns_exceeded', 'ns_met', 'ns_metaabove',
       'ns_nmet', 'ns_notmet'],
      dtype='object')

In [ ]:
for i on df_math['District Code']
    

In [120]:
#df_math_g3.to_csv (r'math_g3_district.csv', index = False, header=True)

In [141]:
#df_math_g4=df_math[df_math['Grade']==4]
#df_math_g4

In [142]:
#df_math_g5=df_math[df_math['Grade']==5]
#df_math_g5

In [143]:
#df_math_g6=df_math[df_math['Grade']==6]
#df_math_g6

In [144]:
#df_math_g7=df_math[df_math['Grade']==7]
#df_math_g7

In [145]:
#df_math_g8=df_math[df_math['Grade']==8]
#df_math_g8

In [146]:
#df_math_g11=df_math[df_math['Grade']==11]
#df_math_g11

In [147]:
#df_math_g13=df_math[df_math['Grade']==13]
#df_math_g13